In [1]:
import transformers

import torch
import torch.nn.functional as F
from torch import nn
from torch.cuda.amp import custom_fwd, custom_bwd

from bitsandbytes.functional import quantize_blockwise, dequantize_blockwise

from tqdm.auto import tqdm

In [2]:
class FrozenBNBLinear(nn.Module):
    def __init__(self, weight, absmax, code, bias=None):
        assert isinstance(bias, nn.Parameter) or bias is None
        super().__init__()
        self.out_features, self.in_features = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
        self.bias = bias
 
    def forward(self, input):
        output = DequantizeAndLinear.apply(input, self.weight, self.absmax, self.code, self.bias)
        if self.adapter:
            output += self.adapter(input)
        return output
 
    @classmethod
    def from_linear(cls, linear: nn.Linear) -> "FrozenBNBLinear":
        weights_int8, state = quantize_blockise_lowmemory(linear.weight)
        return cls(weights_int8, *state, linear.bias)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.in_features}, {self.out_features})"
 
 
class DequantizeAndLinear(torch.autograd.Function): 
    @staticmethod
    @custom_fwd
    def forward(ctx, input: torch.Tensor, weights_quantized: torch.ByteTensor,
                absmax: torch.FloatTensor, code: torch.FloatTensor, bias: torch.FloatTensor):
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        ctx.save_for_backward(input, weights_quantized, absmax, code)
        ctx._has_bias = bias is not None
        return F.linear(input, weights_deq, bias)
 
    @staticmethod
    @custom_bwd
    def backward(ctx, grad_output: torch.Tensor):
        assert not ctx.needs_input_grad[1] and not ctx.needs_input_grad[2] and not ctx.needs_input_grad[3]
        input, weights_quantized, absmax, code = ctx.saved_tensors
        # grad_output: [*batch, out_features]
        weights_deq = dequantize_blockwise(weights_quantized, absmax=absmax, code=code)
        grad_input = grad_output @ weights_deq
        grad_bias = grad_output.flatten(0, -2).sum(dim=0) if ctx._has_bias else None
        return grad_input, None, None, None, grad_bias
 
 
class FrozenBNBEmbedding(nn.Module):
    def __init__(self, weight, absmax, code):
        super().__init__()
        self.num_embeddings, self.embedding_dim = weight.shape
        self.register_buffer("weight", weight.requires_grad_(False))
        self.register_buffer("absmax", absmax.requires_grad_(False))
        self.register_buffer("code", code.requires_grad_(False))
        self.adapter = None
 
    def forward(self, input, **kwargs):
        with torch.no_grad():
            # note: both quantuized weights and input indices are *not* differentiable
            weight_deq = dequantize_blockwise(self.weight, absmax=self.absmax, code=self.code)
            output = F.embedding(input, weight_deq, **kwargs)
        if self.adapter:
            output += self.adapter(input)
        return output 
 
    @classmethod
    def from_embedding(cls, embedding: nn.Embedding) -> "FrozenBNBEmbedding":
        weights_int8, state = quantize_blockise_lowmemory(embedding.weight)
        return cls(weights_int8, *state)
 
    def __repr__(self):
        return f"{self.__class__.__name__}({self.num_embeddings}, {self.embedding_dim})"
 
 
def quantize_blockise_lowmemory(matrix: torch.Tensor, chunk_size: int = 2 ** 20):
    assert chunk_size % 4096 == 0
    code = None
    chunks = []
    absmaxes = []
    flat_tensor = matrix.view(-1)
    for i in range((matrix.numel() - 1) // chunk_size + 1):
        input_chunk = flat_tensor[i * chunk_size: (i + 1) * chunk_size].clone()
        quantized_chunk, (absmax_chunk, code) = quantize_blockwise(input_chunk, code=code)
        chunks.append(quantized_chunk)
        absmaxes.append(absmax_chunk)
 
    matrix_i8 = torch.cat(chunks).reshape_as(matrix)
    absmax = torch.cat(absmaxes)
    return matrix_i8, (absmax, code)
 
 
def convert_to_int8(model):
    """Convert linear and embedding modules to 8-bit with optional adapters"""
    for module in list(model.modules()):
        for name, child in module.named_children():
            if isinstance(child, nn.Linear):
                print(name, child)
                setattr( 
                    module,
                    name,
                    FrozenBNBLinear(
                        weight=torch.zeros(child.out_features, child.in_features, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                        bias=child.bias,
                    ),
                )
            elif isinstance(child, nn.Embedding):
                setattr(
                    module,
                    name,
                    FrozenBNBEmbedding(
                        weight=torch.zeros(child.num_embeddings, child.embedding_dim, dtype=torch.uint8),
                        absmax=torch.zeros((child.weight.numel() - 1) // 4096 + 1),
                        code=torch.zeros(256),
                    )
                )

In [3]:
class GPTJBlock(transformers.models.gptj.modeling_gptj.GPTJBlock):
    def __init__(self, config):
        super().__init__(config)

        convert_to_int8(self.attn)
        convert_to_int8(self.mlp)


class GPTJModel(transformers.models.gptj.modeling_gptj.GPTJModel):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)
        

class GPTJForCausalLM(transformers.models.gptj.modeling_gptj.GPTJForCausalLM):
    def __init__(self, config):
        super().__init__(config)
        convert_to_int8(self)


transformers.models.gptj.modeling_gptj.GPTJBlock = GPTJBlock  # monkey-patch GPT-J

In [4]:
config = transformers.GPTJConfig.from_pretrained("EleutherAI/gpt-j-6B")
tokenizer = transformers.AutoTokenizer.from_pretrained("EleutherAI/gpt-j-6B")

Downloading:   0%|          | 0.00/930 [00:00<?, ?B/s]

In [5]:
gpt = GPTJForCausalLM.from_pretrained("hivemind/gpt-j-6B-8bit", low_cpu_mem_usage=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
_ = gpt.to(device)

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, bias=False)
fc_in Linear(in_features=4096, out_features=16384, bias=True)
fc_out Linear(in_features=16384, out_features=4096, bias=True)
k_proj Linear(in_features=4096, out_features=4096, bias=False)
v_proj Linear(in_features=4096, out_features=4096, bias=False)
q_proj Linear(in_features=4096, out_features=4096, bias=False)
out_proj Linear(in_features=4096, out_features=4096, 

In [6]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(gpt)

861408

In [7]:
device = 'cuda'
_ = gpt.to(device)

In [13]:
%%time

text = f"""
A garbled conversation is transcribed as follows:
Left: THIS IS MIKE FROM QB EINSURANCE HOW CAN I HELP YOU
Right: NOT TO BAD NOT TO BAD HOW ARE YOU
Left: ALL IS GOOD DO YOU HAVE A POLICY NUMBER

Please correct the conversation and write it below, in lower case and adding punctuation:
""".strip()

prompt = tokenizer(text, return_tensors='pt')
prompt = {key: value.to(device) for key, value in prompt.items()}
out = gpt.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 50, do_sample=False)
print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A garbled conversation is transcribed as follows:
Left: THIS IS MIKE FROM QB EINSURANCE HOW CAN I HELP YOU
Right: NOT TO BAD NOT TO BAD HOW ARE YOU
Left: ALL IS GOOD DO YOU HAVE A POLICY NUMBER

Please rewrite the conversation in lower case:

THIS IS MIKE FROM QB EINSURANCE HOW CAN I HELP YOU
NOT TO BAD NOT TO BAD HOW ARE YOU
ALL IS GOOD DO YOU HAVE A POLICY NUMBER

The following is a garbled conversation:
CPU times: user 8.66 s, sys: 15.6 ms, total: 8.67 s
Wall time: 8.67 s


In [128]:
%%time

statement = "The earth is flat"
text = f"""
A person is trying to decide if a statement makes sense. 

Prior examples are:
* Does "skyscrapers are part of shopping list" make sense? No.
* Does "batteries go well with fish" make sense? No.
* Does "Pasta is an Italian dish" make sense? Yes.
* Does "The earth is flat" make sense? No.


Using the examples above, does "{statement}" make sense?
""".strip()

prompt = tokenizer(text, return_tensors='pt')
prompt = {key: value.to(device) for key, value in prompt.items()}
out = gpt.generate(**prompt, max_length=prompt['input_ids'].shape[1] + 5, do_sample=False)
print(tokenizer.decode(out[0]))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


A person is trying to decide if a statement makes sense. 

Prior examples are:
* Does "skyscrapers are part of shopping list" make sense? No.
* Does "batteries go well with fish" make sense? No.
* Does "Pasta is an Italian dish" make sense? Yes.
* Does "The earth is flat" make sense? No.


Using the examples above, does "The earth is flat" make sense?

Yes.

CPU times: user 1 s, sys: 0 ns, total: 1 s
Wall time: 1e+03 ms


## Testing

In [31]:
import json

In [32]:
dev_list = json.load(open('../data/creak_dev_list.json'))

In [23]:
import sys
import traceback

def test(model, data):
    model.eval()
    tp = 0
    fp = 0
    fn = 0

    skipped = 0

    for item in tqdm(data):
        expected = get_answer_from_text(item)
        predicted = ''
        try:
            predicted = generate_answer(model, item)
        except (IndexError, RuntimeError) as e:
            skipped += 1
            continue

        if expected == predicted:
            tp += 1
        if expected == 'N' and predicted == 'Y':
            fp += 1
        if expected == 'Y' and predicted == 'N':
            fn += 1

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1:', f1)
    print('Skipped:', skipped)

In [43]:
def get_text_up_to_question(text):
    _claim_yn = 'The claim makes sense:\n'
    return text[:text.find(_claim_yn) + len(_claim_yn)]

In [86]:
def get_text_up_to_question_for_generation(text):
    _claim_yn = 'The claim makes sense:\n'
    _new_claim_yn = 'Does the claim make sense:\n'
    text = text.replace(_claim_yn, _new_claim_yn)
    pos = text.find(_new_claim_yn) + len(_new_claim_yn)
    return text[:pos]

In [87]:
def get_answer_from_text(text):
    _claim_yn = 'The claim makes sense:\n'
    pos = text.find(_claim_yn) + len(_claim_yn)
    return text[pos]

In [88]:
def get_answer_from_text_for_generation(text):
    _claim_yn = 'The claim makes sense:\n'
    _new_claim_yn = 'Does the claim make sense:\n'
    text = text.replace(_claim_yn, _new_claim_yn)
    pos = text.find(_new_claim_yn) + len(_new_claim_yn)
    return text[pos + 1]

In [70]:
def generate_answer(model, text):
    prompt = get_text_up_to_question_for_generation(text)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 2
    tokens_length = tokens.shape[1]
    if tokens_length + _length >= 1024:
        raise RuntimeError('Text is longer than 1024')
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length, 
             pad_token_id=50256
    )
    output = tokenizer.decode(output[0], skip_special_tokens=True)
    return get_answer_from_text_for_generation(output)

In [94]:
def generate_full_answer(model, text):
    prompt = get_text_up_to_question_for_generation(text)
    tokens = tokenizer.encode(prompt, return_tensors='pt')
    _length = 20
    tokens_length = tokens.shape[1]
    if tokens_length + _length >= 1024:
        raise RuntimeError('Text is longer than 1024')
    output = model.generate(
             tokens.cuda(),
             max_length=tokens_length + _length, 
             pad_token_id=50256
    )
    score = model(output, labels=output)[0]
    out_text = tokenizer.decode(output[0][tokens_length:], skip_special_tokens=True)

    return out_text, float(score)

In [28]:
model = gpt
model.cuda()

GPTJForCausalLM(
  (transformer): GPTJModel(
    (wte): FrozenBNBEmbedding(50400, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (h): ModuleList(
      (0): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0.0, inplace=False)
          (k_proj): FrozenBNBLinear(4096, 4096)
          (v_proj): FrozenBNBLinear(4096, 4096)
          (q_proj): FrozenBNBLinear(4096, 4096)
          (out_proj): FrozenBNBLinear(4096, 4096)
        )
        (mlp): GPTJMLP(
          (fc_in): FrozenBNBLinear(4096, 16384)
          (fc_out): FrozenBNBLinear(16384, 4096)
          (dropout): Dropout(p=0.0, inplace=False)
        )
      )
      (1): GPTJBlock(
        (ln_1): LayerNorm((4096,), eps=1e-05, elementwise_affine=True)
        (attn): GPTJAttention(
          (attn_dropout): Dropout(p=0.0, inplace=False)
          (resid_dropout): Dropout(p=0

In [90]:
get_answer_from_text(dev_list[1])

'N'

In [91]:
generate_answer(model, dev_list[1])

'Y'

In [96]:
print(dev_list[2])

The claim is:
Peter Sellers spent his life as a single bachelor.

The claim makes sense:
Nope.

Because:
He was married four times with three children.


In [98]:
%%time

generate_full_answer(model, dev_list[2])

CPU times: user 3.59 s, sys: 0 ns, total: 3.59 s
Wall time: 3.59 s


("\nYes, it does.\n\nNo, it doesn't.\n\nI think it does", 2.367492914199829)

In [72]:
_ = model.eval()
test(model, dev_list)

  0%|          | 0/1371 [00:00<?, ?it/s]

Precision: 0.47237880496054113
Recall: 0.9976190476190476
F1: 0.6411629686304514
Skipped: 0


In [83]:
story = "skyscrapers can be part of a shopping list"

text = f"""
The claim is:
{story}

The claim makes sense:
"""[1:]


In [82]:
story = '"please add pasta" is more of a statement than an item of a shopping list.'

text = f"""
The claim is:
{story}

The claim makes sense:
"""[1:]


In [84]:
print(text)

The claim is:
skyscrapers can be part of a shopping list

The claim makes sense:



In [85]:
print(generate_full_answer(model, text)[0])


Yes, skyscrapers can be part of a shopping list.

The claim is:
skyscrapers can be part of a shopping list

The claim makes sense (yes or no):

Yes, skyscrapers can be part of a shopping list.

The claim is:
skyscrapers can
